## Intro

This is a silly project that started with me and my brothers watching overwatch league matches and complaining about how terrible the casters are at filling time. I was looking for a new data science personal project and figured why not try and solve our problem with machine learning. In this notebook, I've written a convolutiuon neurel network to identify gameplay and skip all other parts of the match. It actually works pretty well.

Things I read while doing this

1) http://zulko.github.io/blog/2014/07/04/automatic-soccer-highlights-compilations-with-python/

2) https://www.udemy.com/course/machinelearning/learn/lecture/6761136

3) https://www.toptal.com/machine-learning/machine-learning-video-analysis

4) https://www.analyticsvidhya.com/blog/2018/09/deep-learning-video-classification-python/


In [3]:
import cv2
import math
import pandas as pd
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
import tensorflow as tf
from moviepy.editor import VideoFileClip, concatenate
from keras.preprocessing.image import ImageDataGenerator

## Cutting Up Train Videos Into Frames

In [221]:
# count = 0
# videoFile = '@Chengdu Hunters vs @Dallas Fuel _ Summer Showdown Tournament _ Day 1-5kVvsQYk2I0.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/1_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

Done!


In [ ]:
# count = 0
# videoFile = 'Losers Semi-Final _ @ATL Reign   vs   @Dallas Fuel _ Summer Showdown Tournament _ Day 2-RKKXYefOz5w.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/2_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

In [ ]:
# count = 0
# videoFile = 'Winners Finals _ @Chengdu Hunters vs  @Shanghai Dragons  _ Summer Showdown Tournament _ Day 2-hyCOml7JcrA.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/3_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

From here, I manually sorted as gameplay vs not gameplay

## Loading In Video to Process

In [224]:
count = 0
videoFile = 'Winners Finals _ @Chengdu Hunters vs  @Shanghai Dragons  _ Summer Showdown Tournament _ Day 2-hyCOml7JcrA.mp4'
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)
test_imgID = []
while(cap.isOpened()):
    frameId = cap.get(1)
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="pred/3_train%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
        test_imgID.append(filename)
cap.release()
print ("Done!")

Done!


## Data Preprocessing

In [4]:
image_generator = ImageDataGenerator(rescale = 1./255,
                                   validation_split=.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = image_generator.flow_from_directory(batch_size=32,
                                                 directory='train',
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="training",
                                                 class_mode='binary')
test_set = image_generator.flow_from_directory(batch_size=32,
                                                 directory='train',
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="validation",
                                                 class_mode='binary')

Found 12598 images belonging to 2 classes.
Found 3149 images belonging to 2 classes.


## Building the CNN

In [5]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
394/394 [==============================] - 153s 388ms/step - loss: 0.0844 - accuracy: 0.9722 - val_loss: 0.0484 - val_accuracy: 0.9889
Epoch 2/10
394/394 [==============================] - 134s 340ms/step - loss: 0.0308 - accuracy: 0.9916 - val_loss: 0.0444 - val_accuracy: 0.9838
Epoch 3/10
394/394 [==============================] - 132s 336ms/step - loss: 0.0255 - accuracy: 0.9925 - val_loss: 0.0245 - val_accuracy: 0.9917
Epoch 4/10
394/394 [==============================] - 131s 333ms/step - loss: 0.0141 - accuracy: 0.9956 - val_loss: 0.0384 - val_accuracy: 0.9879
Epoch 5/10
394/394 [==============================] - 132s 334ms/step - loss: 0.0143 - accuracy: 0.9960 - val_loss: 0.0286 - val_accuracy: 0.9898
Epoch 6/10
394/394 [==============================] - 131s 334ms/step - loss: 0.0133 - accuracy: 0.9956 - val_loss: 0.0368 - val_accuracy: 0.9835
Epoch 7/10
394/394 [==============================] - 132s 334ms/step - loss: 0.0107 - accuracy: 0.9963 - val_loss: 0.0474 -

## Preprocessing Test Data

In [27]:
test_image = []
for img_name in test_imgID:
    img = plt.imread('pred/' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [31]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(64,64)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

## Predictions

In [32]:
predictions = (cnn.predict(test_image)>0.5).astype("int32")

In [61]:
training_set.class_indices

{'ad': 0, 'not_ad': 1}

In [33]:
predictions.shape

(9531, 1)

In [39]:
pred=(predictions.ravel())

In [38]:
predictions

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [36]:
np.savetxt("pred.csv", predictions,delimiter=",")

So, I want some sort of averaging here. Rolling average 

In [91]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [151]:
pred= moving_average(predictions.ravel(), 10)

In [202]:
pred=(pred>0.5).astype("int32")

In [105]:
pred=predictions.ravel()
sum(pred)

4404

## Turning Predictions Into Something Workable

In [117]:
test_pred = (moving_average(pred,60)>.5).astype("int32")

In [118]:
count = 0
times = []
for i in range(0,len(test_pred)):
    if test_pred[i] == 1:
        times.append(i)

In [119]:
times

[207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373

In [121]:
count = 0
start_stop = []
for i in range(0,len(times)-1):
    if times[i] == times[i+1]-1:
        count = count + 1
    else:
        ft.append((times[i]-count,times[i]))
        count = 0 
start_stop

[(207, 606),
 (651, 1004),
 (1011, 1029),
 (1037, 1059),
 (1092, 1318),
 (1838, 2239),
 (2247, 2780),
 (3432, 3619),
 (3627, 3635),
 (3647, 3919),
 (3981, 3981),
 (4003, 4035),
 (4037, 4070),
 (4091, 4210),
 (4218, 4221),
 (4232, 4513),
 (4535, 4621),
 (4633, 4643),
 (4648, 4655),
 (4726, 4819),
 (4824, 4831),
 (5397, 6579),
 (7097, 7114),
 (7755, 8196),
 (8286, 8333),
 (9129, 9142)]

So the start and stop times are much more readable, but some of these should be part of the same section. 207-606 then 651 - 1004... etc. Those should all be in one section from 207-1318

In [122]:
reduced = [start_stop[0]]
for t in start_stop:
    if t[0] - reduced[-1][1] < 180:
        reduced[-1] = (reduced[-1][0],t[1])
    else:
        reduced.append(t)
reduced

[(207, 1318),
 (1838, 2780),
 (3432, 4831),
 (5397, 6579),
 (7097, 7114),
 (7755, 8333),
 (9129, 9142)]

Looking up above we are doing much better, but we have a clip with tiny length, the last start/stop pair is too small so lets make a section of code that clears anything less than 30 seconds (highlight video length)

In [126]:
final_time = []
for item in reduced:
    if item[1]-item[0] > 30:
        final_time.append(item)
final_time

[(207, 1318), (1838, 2780), (3432, 4831), (5397, 6579), (7755, 8333)]

## Writting Out Movie

In [129]:
clip = VideoFileClip('Winners Finals _ @Chengdu Hunters vs  @Shanghai Dragons  _ Summer Showdown Tournament _ Day 2-hyCOml7JcrA.mp4')
final = concatenate([clip.subclip(max(ft3[i][0]-30,0), min(ft3[i][1]+30,clip.duration)) for i in range(0,len(ft3))])
final.to_videofile('test2.mp4')

Moviepy - Building video test2.mp4.
MoviePy - Writing audio in test2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test2.mp4



Moviepy - Done !
Moviepy - video ready test2.mp4


## Next Steps

1) Setup image generator for the predictions. Right now this takes an ungodly amount of ram

2) Make a callout for youtube.dl to run everything from this one notebook

3) Add more trainning Data

4) Increase the number of Epochs for the CNN